### Bokeh tutorial

Bokeh is a **visualisation library**, that can be used, among other things, to create interactive plots. 

Below is a collection of examples with accompanying explanations to ease the transition from "common" libraries such as **Seaborn** and **Matplotlib** to an interactive one.

Other interactive plotting tools include **Plotly** and **Plotly Express**, but these are outside the scope of the current tutorial.


Before we begin with the worked example, we must update our library.

Here I will demonstrate using Conda, but it is also possible with pip or through the terminal; please refer to the [Bokeh documentation](https://docs.bokeh.org/en/latest/docs/first_steps/installation.html#installation) for further details.

1. From the terminal (making sure you are in your environment of choice) enter the following command: `conda install -c bokeh bokeh`. This will ensure you are installing the latest version.

2. Once complete, you can enter this code in the terminal in order to download the sample data: `bokeh sampledata`

We are now ready to run the worked example in the updated Python environment.


In [6]:
import pandas as pd
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.sampledata.stocks import AAPL, GOOG, IBM, MSFT

As stated, this tutorial will leverage the stock database that bokeh has available. 
It imports a dictionary for each stock requested, representing a time series for the price of that particular stock. 

We will use Apple, Google, IBM and Microsoft stocks.

In [7]:
# First, we'll create pandas dataframes from the dictionaries, for ease of use
apple=pd.DataFrame.from_dict(AAPL)
google=pd.DataFrame.from_dict(GOOG)
ibm=pd.DataFrame.from_dict(IBM)
micro=pd.DataFrame.from_dict(MSFT)

# Have a look at what the dataframes contain:
print(apple.head(5))

# After checking, we see that the date column is not of datetime type.
# As we work with time series, we need to convert it to datetime using pandas.
apple["date"]=pd.to_datetime(apple["date"])
google["date"]=pd.to_datetime(google["date"])
ibm["date"]=pd.to_datetime(ibm["date"])
micro["date"]=pd.to_datetime(micro["date"])

         date    open    high     low   close    volume  adj_close
0  2000-03-01  118.56  132.06  118.50  130.31  38478000      31.68
1  2000-03-02  127.00  127.94  120.69  122.00  11136800      29.66
2  2000-03-03  124.87  128.23  120.00  128.00  11565200      31.12
3  2000-03-06  126.00  129.13  125.00  125.69   7520000      30.56
4  2000-03-07  126.44  127.44  121.12  122.87   9767600      29.87


Now we'll start plotting. Apart from the interactive section, graphing in bokeh
follows similar logic to the graphing libraries we're used to.

This will open our graph in a new browser tab rather than directly in our code editor.

In [8]:
# First, we create a figure which we will plot on top of.
fig=figure(width=1600, height=500, x_axis_type="datetime")
# It's necessary to include (x_axis_type="datetime") when working with time series.

# We can change the title, labels etc. directly on the figure.
fig.title.text="Our title"
fig.xaxis.axis_label="Our x-axis label"
fig.yaxis.axis_label="Our y-axis label"

# Now, before we can plot our stock, we need to "transform" our data 
# from a pandas dataframe to a ColumnDataSource (CDS) file which is
# readable by bokeh.
transformed_apple=ColumnDataSource(apple)
fig.line("date", "open", source=transformed_apple)
show(fig)

Great! We created our first plot; but such a graph could have been made in other libraries as well. What about the interactions? By default we can only drag and move the graph. Let's delve a little deeper and try to implement a zooming in/out mechanism.

In [9]:
# Once more we create the figure, customize our title, labels etc.
# But now we will pass some tools into the figure

our_tools="pan", "wheel_zoom" 
            # Two extremely common tools, they are included in the graphs
            # by default. Added them to showcase tool addition.
            # For more tool selection see the documentation: 
            # https://docs.bokeh.org/en/2.4.0/docs/reference/models/tools.html

fig1=figure(width=1600, height=500, tools=our_tools,
            active_scroll="wheel_zoom",
            x_axis_type="datetime")
            # active_scroll defines which action will zoom in/out

fig1.title.text="Our interactive title"
fig1.xaxis.axis_label="Our interactive x-axis label"
fig1.yaxis.axis_label="Our interactive y-axis label"

transformed_apple=ColumnDataSource(apple)
fig1.line("date", "open", source=transformed_apple)
show(fig1)

Ok, now we can zoom in/out, pan etc. But a stock price graph like that, spanning more than a decade, can make distinguishing stock price for a particular date difficult. 

In addition, our datasets contain information about opening and closing prices, trading volumes etc. which is not currently visible. We should include them in such a way that mousing over the graph shows us the details for the corresponding date.

In [10]:
# We begin as usual
our_tools="pan", "wheel_zoom"
fig2=figure(width=1600, height=500, tools=our_tools,
                active_scroll="wheel_zoom",
                x_axis_type="datetime")
fig2.title.text="Our intersecting title"
fig2.xaxis.axis_label="Our intersecting x-axis label"
fig2.yaxis.axis_label="Our intersecting y-axis label"
transformed_apple=ColumnDataSource(apple)
fig2.line("date", "open", source=transformed_apple)

#Now we add our more advanced tools
fig2.add_tools(HoverTool(tooltips=[
                                ("Calendar date", "@date{%F}"), 
                                # The {%F} "marks" the variable for formatting
                                ("Opening price", "@open{0,}"),
                                # The {0,} denotes that we want numbers with
                                # commas denoting thousands, millions etc.
                                ("Daily high", "@high{0,}"),
                                ("Daily low", "@low{0,}"),
                                ("Closing price", "@close{0,}"),
                                ("Trading volume", "@volume{0,}")
                                ],
                        formatters={"@date": "datetime"},
                        mode="vline")
                )
show(fig2)

Perfect! Now we have all the information about a stock at any given date. But we have data for four different stocks available. Wouldn't it be better to display them all and switch on and off whichever ones we want?

In [11]:
# The process of doing this is completely straightforward
our_tools="pan", "wheel_zoom"
fig3=figure(width=1600, height=500, tools=our_tools,
                active_scroll="wheel_zoom",
                x_axis_type="datetime")
fig3.title.text="Our combined title"
fig3.xaxis.axis_label="Our combined x-axis label"
fig3.yaxis.axis_label="Our combined y-axis label"

#Now we pass the graph for each stock, as we have done already
transformed_apple=ColumnDataSource(apple)
fig3.line("date", "open", source=transformed_apple,
                line_width=1.7, alpha=1,
                color="crimson", legend_label="Apple")
                # We need to define the colours and legends ourselves, otherwise
                # we won't be able to differeniate between them
transformed_google=ColumnDataSource(google)
fig3.line("date", "open", source=transformed_google,
                line_width=1.7,
                color="olive", legend_label="Google")
transformed_ibm=ColumnDataSource(ibm)
fig3.line("date", "open", source=transformed_ibm,
                line_width=1.7,
                color="fuchsia", legend_label="IBM")
transformed_micro=ColumnDataSource(micro)
fig3.line("date", "open", source=transformed_micro,
                line_width=1.7,
                color="black", legend_label="Microsoft")

# Add our tools as per usual
fig3.add_tools(HoverTool(tooltips=[
                                ("Calendar date", "@date{%F}"), 
                                ("Opening price", "@open{0,}"),
                                ("Daily high", "@high{0,}"),
                                ("Daily low", "@low{0,}"),
                                ("Closing price", "@close{0,}"),
                                ("Trading volume", "@volume{0,}")
                                ],
                        formatters={"@date": "datetime"},
                        mode="vline")
                )
show(fig3)

This is great progress, but the hover tooltips of all the stocks overlap, which makes it difficult to make sense of the screen. We want the ability to "mute" stocks at will, so we can view the desirable ones at our leisure.

In [12]:
# The code is virtually the same ass before, with some minor changes
our_tools="pan", "wheel_zoom"
fig4=figure(width=1600, height=500, tools=our_tools,
                active_scroll="wheel_zoom",
                x_axis_type="datetime")
fig4.title.text="Our final title"
fig4.xaxis.axis_label="Our final x-axis label"
fig4.yaxis.axis_label="Our final y-axis label"


transformed_apple=ColumnDataSource(apple)
fig4.line("date", "open", source=transformed_apple,
                line_width=1.7, color="crimson", alpha=1,
                # Alpha denotes how visible our line will be, in range[0,1]
                muted_color="crimson", muted_alpha=0.2,
                # The muted_arguments define the colour and visibility 
                # of the lines once muted
                legend_label="Apple")
transformed_google=ColumnDataSource(google)
fig4.line("date", "open", source=transformed_google,
                line_width=1.7, color="olive", alpha=1,
                muted_color="olive", muted_alpha=0.2,
                legend_label="Google")
transformed_ibm=ColumnDataSource(ibm)
fig4.line("date", "open", source=transformed_ibm,
                line_width=1.7, color="fuchsia", alpha=1,
                muted_color="fuchsia", muted_alpha=0.2,
                legend_label="IBM")
transformed_micro=ColumnDataSource(micro)
fig4.line("date", "open", source=transformed_micro,
                line_width=1.7, color="black", alpha=1,
                muted_color="black", muted_alpha=0.2,
                legend_label="Microsoft")


fig4.add_tools(HoverTool(tooltips=[
                                ("Calendar date", "@date{%F}"), 
                                ("Opening price", "@open{0,}"),
                                ("Daily high", "@high{0,}"),
                                ("Daily low", "@low{0,}"),
                                ("Closing price", "@close{0,}"),
                                ("Trading volume", "@volume{0,}")
                                ],
                        formatters={"@date": "datetime"},
                        muted_policy="ignore", # Default is show, we set it to
                                        # ignore so that there won't be
                                        # hover tooltips on muted lines
                        mode="vline")
                )
fig4.legend.location="top_left"
fig4.legend.click_policy="mute" # On clicking the legend, the corresponding
                        # line will become less visible. If we pass "hide" as
                        # an argument, it will disappear completely
show(fig4)

Success! We have created an interactive stock graph, like those we see at MarketWatch and Bloomberg.